# **Car Evaluation Analysis**

### **Introduction**

**Background**: 

Cars and personal transportation are an inevitable part of everyday life in the developed world. They play a crucial role in people's daily routines, enabling them to commute to work, travel, attend social gatherings, and explore new places. However, cars also pose significant safety risks. In 2023 alone, car accidents accounted for over 40,000 fatalities. Given these risks, it is no surprise that many individuals seek ways to make car travel safer while maintaining its convenience and necessity.

Research has shown that consumers are willing to invest in vehicle safety. For example, Andersson (2008) found that Swedish drivers were willing to pay a premium for improved car design and build quality to reduce the risk of injury in accidents. Additionally, design and material choices play a critical role in determining a vehicle's safety. Nygren (1983) found that factors such as a car’s weight, seatbelt design, and headrests significantly influenced accident survivability. More recently, Richter et al. (2005) demonstrated that passive safety improvements—such as enhanced structural integrity and interior design modifications—have contributed to a measurable decline in injury rates from car accidents. These findings highlight the importance of identifying key factors that contribute to vehicle safety.

Given this context, our research project aims to answer the following question:

**Can we predict the estimated safety of a car using various attributes, such as its buying price, capacity, and maintenance cost?**

To answer this question, we will use the Car Evaluation Dataset from UC Irvine’s Machine Learning Repository. This multivariate classification dataset contains six car design and classification variables, and includes 1,728 observations. Key variables that will be central to our analysis include the car’s buying price, maintenance cost, seating capacity (in terms of the number of passengers it can accommodate), and the car’s evaluation level (categorized as unacceptable, acceptable, good, or very good).

### **References**

- Andersson, H. (2008). Willingness to pay for car safety: evidence from Sweden.    Environmental and Resource Economics, 41, 579-594.

- Nygren, Å. (1983). Injuries to car occupants—Some aspects of the interior safety of cars: A study of a five-year material from an insurance company. Acta Oto-Laryngologica, 95(sup395), 1-135.

- Richter, M., Pape, H. C., Otte, D., & Krettek, C. (2005). Improvements in passive car safety led to decreased injury severity–a comparison between the 1970s and 1990s. Injury, 36(4), 484-488.

- Wood, D. P. (1997). Safety and the car size effect: A fundamental explanation. Accident Analysis & Prevention, 29(2), 139-151.

- Wikipedia Contributors. (2019, March 21). Motor vehicle fatality rate in U.S. by year. Wikipedia; Wikimedia Foundation. https://en.wikipedia.org/wiki/Motor_vehicle_fatality_rate_in_U.S._by_year

‌